### Community College Early Learning Credentials Supply and Demand Analysis

The code below performs two general tasks: 

- Prepare 'demand' data. Demand data is the number of child care staff who need additional education in order to meet or exceed the new minimum education requirements for early learning staff members stated in recently passed legislation in Washington State. 
- Generate 'supply' data. Supply data is the number of child care staff to whom a community college can offer an early learning-related credential each year. I have not collected these data yet. For the moment, these data are randomly generated in order to build the analysis framework.

All supply and demand data are aggregated by college and credential type. 

In the dataset there are four colleges and four types of credentials (AA, State Certificate, Short Certificate, and Initial Certificate).

These data are a subset of a larger dataset and are blind.

### Load libraries

In [100]:
import pandas as pd
import numpy as np
from random import randint

### Load Demand Data

I import two datasets below:

- `prov_data` includes Staff ID, child care site ID, and a flag which indicates the type of credential the staff member needs to earn in order to meet the new education standards. Not all staff require additional education.
- `travel_times` includes the amount of time it takes to drive between child care sites and community colleges. Drive times are determined using the Google Distance Matrix API. I 'match' (or assign) staff members to colleges based on the shortest drive time.

In [130]:
prov_data = pd.read_csv('http://goo.gl/pzVrGK')

mr_no = (
    prov_data
    .groupby(by = ['pid', 'credential'])
    .count()
    .reset_index()
    .rename(columns = {'staff_id': 'mr_no'})
    .pivot(index = 'pid', columns = 'credential', values = 'mr_no')
    .reset_index()
    )

travel_times = pd.read_csv('http://goo.gl/e2oxfV')

below_requirements_by_college = (
    pd.melt(
        (travel_times.query('compare_val == True')
         .merge(mr_no)
         .drop(['compare_val'], 1)
        )
        , id_vars=['col_id', 'college', 'pid', 'travel_time']
        , value_vars=['demand_aa_staff', 'demand_state_staff'
                      , 'demand_short_staff', 'demand_initial_staff']
    )
    .groupby(['col_id', 'college', 'variable'])['value']
    .sum()
    .reset_index()
    .pivot_table(index = ['col_id', 'college'], columns = 'variable', values = 'value')
    .reset_index()
    .fillna(0)
    .assign(
        demand_aa_credential = lambda x: x.demand_aa_staff
        , demand_state_credential = lambda x: x.demand_state_staff + x.demand_aa_staff
        , demand_short_credential = lambda x: x.demand_short_staff + x.demand_state_staff + x.demand_aa_staff
        , demand_initial_credential = lambda x: x.demand_initial_staff + x.demand_short_staff + x.demand_state_staff + x.demand_aa_staff
    )
)

### Generate Sample Supply Data

I have not collected supply data yet from colleges. The code below generates data that I use as 'placeholder' supply data until I collect genuine supply data from the community colleges. I use these placeholder data to build an analysis framework.

Also in the cell below, I merge supply and demand data.

The `college_supply_demand` data can be found <a href="https://goo.gl/vWqEBf">here</a>. 

In [132]:
placeholder_supply_data = (
    below_requirements_by_college
    .assign(
        supply_aa_credential = [randint(0,20) for p in range(0,4)]
        , supply_state_credential = [randint(0,100) for p in range(0,4)]
        , supply_short_credential = [randint(0,300) for p in range(0,4)]
        , supply_initial_credential = [randint(0,800) for p in range(0,4)]
    )
    .assign(
        supply_aa_credential = lambda x: x.supply_aa_credential
        , supply_state_credential = lambda x: x.supply_state_credential + x.supply_aa_credential
        , supply_short_credential = lambda x: x.supply_short_credential + x.supply_state_credential
        , supply_initial_credential = lambda x: x.supply_initial_credential + x.supply_short_credential
    )
    .assign(
        supply_aa_staff = lambda x: x.supply_aa_credential
        , supply_state_staff = lambda x: x.supply_state_credential - x.supply_aa_credential
        , supply_short_staff = lambda x: x.supply_short_credential - x.supply_state_credential
        , supply_initial_staff = lambda x: x.supply_initial_credential - x.supply_short_credential
    )
)

college_supply_demand = (
    pd.melt(placeholder_supply_data
            , id_vars=['col_id', 'college']
            , value_vars=['demand_aa_staff', 'demand_state_staff', 'demand_short_staff', 'demand_initial_staff'
                          ,'demand_aa_credential', 'demand_state_credential', 'demand_short_credential', 'demand_initial_credential'
                          , 'supply_aa_staff', 'supply_state_staff', 'supply_short_staff', 'supply_initial_staff'
                          ,'supply_aa_credential', 'supply_state_credential', 'supply_short_credential', 'supply_initial_credential']
           )
    .assign(
        credential = lambda x: np.where(x.variable.str.contains('_aa'), 'AA', 
                                np.where(x.variable.str.contains('_initial'), 'Cert - Initial', 
                                np.where(x.variable.str.contains('_short'), 'Cert - Short',
                                np.where(x.variable.str.contains('_state'), 'Cert - State', None))))
        , level = lambda x: np.where(x.variable.str.contains('_staff'), 'Staff', 'Credentials')
        , sup_demand = lambda x: np.where(x.variable.str.contains('demand'), 'Demand', 'Supply')
    )
    .reset_index()
    .drop(['index', 'col_id', 'variable'], 1)
    .pivot_table(index = ['college', 'level', 'credential'], columns = 'sup_demand', values = 'value')
    .reset_index()
    .rename(columns = {'college': 'College', 'level' : 'Measure', 'credential' : 'Credential'})
)

college_supply_demand.to_csv('college_supply_demand.csv', index = False)